<a href="https://colab.research.google.com/github/cikidano/bm/blob/master/analyze_wifi_pcap_using_scapy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install scapy
from scapy.all import *
!pip install prettytable
from prettytable import PrettyTable

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 30.2 MB/s eta 0:00:00


In [2]:
def filter_packets(packets):
    tcp_packets = packets.filter(lambda p: TCP in p)
    udp_packets = packets.filter(lambda p: UDP in p)
    icmp_packets = packets.filter(lambda p: ICMP in p)
    other_packets = packets.filter(lambda p: not (TCP in p or UDP in p or ICMP in p))
    return tcp_packets, udp_packets, icmp_packets, other_packets

def filter_packets_by_source_ip(packets, source_ip):
    filtered_packets = packets.filter(lambda p: IP in p and p[IP].src == source_ip)
    return filtered_packets

def filter_packets_by_destination_port(packets, dest_port):
    filtered_packets = packets.filter(lambda p: TCP in p and p[TCP].dport == dest_port)
    return filtered_packets

def filter_packets_by_mac_address(packets, mac_address):
    filtered_packets = packets.filter(lambda p: Ether in p and (p.src == mac_address or p.dst == mac_address))
    return filtered_packets

def filter_packets_by_ip_range(packets, start_ip, end_ip):
    filtered_packets = packets.filter(lambda p: IP in p and start_ip <= p[IP].src <= end_ip)
    return filtered_packets

def filter_packets_by_protocol(packets, protocol):
    filtered_packets = packets.filter(lambda p: p.haslayer(protocol))
    return filtered_packets

def parse_dns_packets(packets):
    dns_packets = packets.filter(lambda p: DNS in p)
    domain_info = []
    for packet in dns_packets:
        domain_name = packet[DNS].qd.qname.decode('utf-8')
        domain_info.append(domain_name)
    return domain_info

def filter_suspicious_packets(packets, max_payload_size):
    suspicious_packets = packets.filter(lambda p: TCP in p and len(p[TCP].payload) > max_payload_size)
    return suspicious_packets

def filter_packets_by_tcp_flags(packets, flags):
    filtered_packets = packets.filter(lambda p: TCP in p and p[TCP].flags & flags)
    return filtered_packets

def parse_beacon_frames(packets):
    beacon_frames = packets.filter(lambda p: p.haslayer(Dot11Beacon))
    channel_info = {}
    for beacon in beacon_frames:
        ssid = beacon.info.decode('utf-8', 'ignore')
        bssid = beacon.addr3
        channel = ord(beacon[Dot11Elt:3].info)
        power_constraint = get_power_constraint(beacon)
        if channel not in channel_info:
            channel_info[channel] = []
        channel_info[channel].append((ssid, bssid, power_constraint))
    return channel_info

def get_power_constraint(beacon):
    power_constraint_elem = beacon.getlayer(Dot11Elt, ID=32)
    if power_constraint_elem:
        power_constraint = ord(power_constraint_elem.info)
        return power_constraint
    return None

def parse_management_frames(packets):
    management_frames = packets.filter(lambda p: p.haslayer(Dot11) and p.type == 0)
    frame_info = []
    for frame in management_frames:
        frame_type = frame.subtype
        source_mac = frame.addr2
        destination_mac = frame.addr1
        frame_info.append((frame_type, source_mac, destination_mac))
    return frame_info

def get_frame_type_explanation(frame_type):
    frame_types = {
        0: "Association Request",
        1: "Association Response",
        2: "Reassociation Request",
        3: "Reassociation Response",
        4: "Probe Request",
        5: "Probe Response",
        8: "Beacon",
        9: "ATIM",
        10: "Disassociation",
        11: "Authentication",
        12: "Deauthentication",
        13: "Action"
    }
    return frame_types.get(frame_type, "Unknown")

def summarize_findings(tcp_packets, udp_packets, icmp_packets, other_packets, source_ip_packets, dest_port_packets,
                       mac_address_packets, ip_range_packets, http_packets, suspicious_packets, syn_packets,
                       channel_info, management_frame_info, domain_info):

    table = PrettyTable()
    table.field_names = ["Finding", "Count/Value"]
    table.align["Finding"] = "l"
    table.align["Count/Value"] = "c"

    table.add_row(["TCP Packets", len(tcp_packets)])
    table.add_row(["UDP Packets", len(udp_packets)])
    table.add_row(["ICMP Packets", len(icmp_packets)])
    table.add_row(["Other Packets", len(other_packets)])
    table.add_row(["Packets from Source IP 192.168.0.1", len(source_ip_packets)])
    table.add_row(["Packets to Destination Port 80", len(dest_port_packets)])
    table.add_row(["Packets from/to MAC 00:11:22:33:44:55", len(mac_address_packets)])
    table.add_row(["Packets in IP Range 192.168.0.1 - 192.168.0.100", len(ip_range_packets)])
    table.add_row(["HTTP Packets", len(http_packets)])
    table.add_row(["Suspicious Packets (Payload > 1024)", len(suspicious_packets)])
    table.add_row(["SYN Packets", len(syn_packets)])

    beacon_channels = ", ".join(str(channel) for channel in channel_info.keys())
    table.add_row(["Beacon Frame Channels", beacon_channels])

    for frame_type, source_mac, destination_mac in management_frame_info:
        frame_type_explanation = get_frame_type_explanation(frame_type)
        table.add_row([f"Management Frame: {frame_type_explanation}", ""])

    dns_domains = "\n".join(domain_info)
    table.add_row(["DNS Domains", dns_domains])

    return table

def analyze_pcap(pcap_file):
    """
    Analyzes a pcap file and performs various packet filtering and parsing operations.

    Args:
        pcap_file: The path to the pcap file to analyze.
    """
    target_mac = "c0:7a:d6:9c:91:1f"

    packets = rdpcap(pcap_file)
    tcp_packets, udp_packets, icmp_packets, other_packets = filter_packets(packets)
    source_ip_packets = filter_packets_by_source_ip(packets, "192.168.0.1")
    dest_port_packets = filter_packets_by_destination_port(packets, 80)
    mac_address_packets = filter_packets_by_mac_address(packets, target_mac)
    ip_range_packets = filter_packets_by_ip_range(packets, "192.168.0.1", "192.168.0.100")
    http_packets = filter_packets_by_protocol(packets, TCP)
    domain_info = parse_dns_packets(packets)
    suspicious_packets = filter_suspicious_packets(packets, 1024)
    syn_packets = filter_packets_by_tcp_flags(packets, 0x02)  # SYN flag
    channel_info = parse_beacon_frames(packets)
    management_frame_info = parse_management_frames(packets)

    print("Detailed Output:")
    print(f"Number of TCP packets: {len(tcp_packets)}")
    print(f"Number of UDP packets: {len(udp_packets)}")
    print(f"Number of ICMP packets: {len(icmp_packets)}")
    print(f"Number of other packets: {len(other_packets)}")
    print(f"Packets from source IP 192.168.0.1: {len(source_ip_packets)}")
    print(f"Packets to destination port 80: {len(dest_port_packets)}")
    print(f"Packets from/to MAC address {target_mac}: {len(mac_address_packets)}")
    print(f"Packets in IP range 192.168.0.1 - 192.168.0.100: {len(ip_range_packets)}")
    print(f"HTTP packets: {len(http_packets)}")
    print(f"Suspicious packets (payload size > 1024): {len(suspicious_packets)}")
    print(f"SYN packets: {len(syn_packets)}")

    print("\nBeacon frame information by channel:")
    for channel, beacons in channel_info.items():
        print(f"Channel {channel}:")
        for ssid, bssid, power_constraint in beacons:
            print(f"  SSID: {ssid}, BSSID: {bssid}, Power Constraint: {power_constraint} dBm")

    print("\nManagement frame information:")
    for frame_type, source_mac, destination_mac in management_frame_info:
        frame_type_explanation = get_frame_type_explanation(frame_type)
        print(f"  Frame Type: {frame_type} ({frame_type_explanation}), Source MAC: {source_mac}, Destination MAC: {destination_mac}")

    print("\nDomain information from DNS packets:")
    for domain_name in domain_info:
        print(f"  {domain_name}")

    summary_table = summarize_findings(
        tcp_packets, udp_packets, icmp_packets, other_packets,
        source_ip_packets, dest_port_packets, mac_address_packets,
        ip_range_packets, http_packets, suspicious_packets, syn_packets,
        channel_info, management_frame_info, domain_info
    )

    print("\nSummary of Findings:")
    print(summary_table)

def main():
    pcap_file = "capture.pcap"
    analyze_pcap(pcap_file)

In [3]:
if __name__ == "__main__":
    main()

NameError: name 'rdpcap' is not defined